In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import csv
import re
import time
import random

# Set up chrome driver
driver = webdriver.Chrome('C:\\Users\\JeongwooHan\\Desktop\\ChromeDriver\\chromedriver.exe')

# Set up action chains
actions = ActionChains(driver)

# Define function to find the metrics
def find_metrics(driver, wine_dict):

    metric_dict = {"Light": "light_bold",
                   "Smooth": "smooth_tannic",
                   "Dry": "dry_sweet",
                   "Soft": "soft_acidic"}
    
    all_metric_names = driver.find_elements_by_xpath('//tr[@class="tasteStructure__tasteCharacteristic--1rMFl"]/td/div[@class="tasteStructure__property--loYWN"]')
    selected_metric_names = [all_metric_names[i].text for i in range(len(all_metric_names)) if i%2 == 0]
    final_metric_names = [metric_dict[key] for key in selected_metric_names]
    missing_metric_names = list(set(metric_dict.values()) - set(final_metric_names))
    
    all_metrics_prep = driver.find_elements_by_xpath('//div[@class="indicatorBar__meter--2t_YL tasteStructure__progressBar--hjNb2"]/span[@class="indicatorBar__progress--3aXLX"]')
    all_metrics = [float(re.findall('\d*\.?\d+',all_metrics_prep[i].get_attribute('style'))[1]) for i in range(len(all_metrics_prep))]
    
    if len(missing_metric_names) > 0:
        for i in missing_metric_names:
            wine_dict[i] = -1

    for i,j in zip(final_metric_names, all_metrics):
        wine_dict[i] = j
    
    return wine_dict

# Set up CSV
csv_file = open('vivino_wine.csv', 'w', newline = '')
fieldnames = ['winery','wine_type','city','country','wine_name','year','overall_rating','overall_rating_count',\
              'price','light_bold','smooth_tannic','dry_sweet','soft_acidic']
writer = csv.DictWriter(csv_file, fieldnames = fieldnames)
writer.writeheader()

# Bring in the pages that are separated out into about 1000 ish records
column_names = ["Start", "Finish", "Count", "URL"]
df = pd.read_csv("URLs.csv", names = column_names)
all_pages = df.URL.to_list()
all_pages = all_pages[11:23] # Update whenever VPN gets blocked

page_count = 12
for pages in all_pages:

    print("Scraping Page Number: " + str(page_count))
    page_count = page_count + 1
    
    # Go to the page to scrape
    driver.get(pages)
    time.sleep(random.randint(10,12)/10)

    #Scroll to the bottom
    SCROLL_PAUSE_TIME = random.randint(19,21)/10
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Get all urls
    all_urls = driver.find_elements_by_xpath(".//div[@class='explorerPage__results--3wqLw']/div/div")
    one_url = all_urls[0]
    url = one_url.find_element_by_xpath("./div/div/a")
    all_str_urls = [url.find_element_by_xpath("./div/div/a").get_attribute('href') for url in all_urls]

    index = 1
    for url in all_str_urls:

        # Keep index documented
        print("Scraping Item Number: " + str(index))
        index = index + 1

        # Go into each link
        driver.get(url)
        
        # Clean dictionary to create the current row
        wine_dict = {}

        # Pull different elements using xpath
        try:
            winery = driver.find_element_by_xpath('//span[@class="headline"]/a[@class="winery"]').text
        except Exception as e:
            print(e)
            winery = 'NA'
            
        try:
            wine_type = driver.find_element_by_xpath('//span[@class="wineLocationHeader__wineType--14nrC"]').text
        except Exception as e:
            print(e)
            wine_type = 'NA'
            
        try:
            city = driver.find_element_by_xpath('//a[@class="anchor__anchor--3DOSm wineLocationHeader__region--1cbip"]').text
        except Exception as e:
            print(e)
            city = 'NA'
            
        try:
            country = driver.find_element_by_xpath('//a[@class="anchor__anchor--3DOSm wineLocationHeader__country--1RcW2"]').text
        except Exception as e:
            print(e)
            country = 'NA'
            
        try:
            wine_name = driver.find_element_by_xpath('//span[@class="vintage"]/a[@class="wine"]').text
        except Exception as e:
            print(e)
            wine_name = 'NA'
            
        try:
            year_prep = driver.find_element_by_xpath('//span[@class="vintage"]').text
            year = int(re.findall('\d+', year_prep)[-1])
        except Exception as e:
            print(e)
            year = -1
            
        try:
            overall_rating = float(driver.find_element_by_xpath('//div[@class="vivinoRating__averageValue--3Navj"]').text)
        except Exception as e:
            print(e)
            overall_rating = -1
            
        try:
            overall_rating_count_prep = driver.find_element_by_xpath('//div[@class="vivinoRating__caption--3tZeS"]').text
            overall_rating_count = int(re.findall('\d+', overall_rating_count_prep)[0])
        except Exception as e:
            print(e)
            overall_rating_count = -1
            
        try:
            time.sleep(random.randint(5,7)/10)
            price_prep = driver.find_element_by_xpath('//span[@class="purchaseAvailability__currentPrice--3mO4u"]').text
            price = float(re.findall('\d*\.?\d+', price_prep)[0])
        except:
            try:
                price_prep = driver.find_element_by_xpath('//span[@class="purchaseAvailabilityPPC__amount--2_4GT"]').text
                price = float(re.findall('\d*\.?\d+', price_prep)[0])
            except Exception as e:
                print(e)
                price = -1

        # Need some scrolling for the page to load
        try:
            driver.execute_script("window.scrollTo(0, 1080);")
            driver.execute_script("window.scrollTo(0, 1080);")
            driver.execute_script("window.scrollTo(0, 1080);")
            driver.execute_script("window.scrollTo(0, 1080);")
            driver.execute_script("window.scrollTo(0, 1080);")
            time.sleep(random.randint(20,22)/10)
            wine_dict = find_metrics(driver, wine_dict)
        except Exception as e:
            print(e)
            wine_dict['light_bold'] = -1
            wine_dict['smooth_tannic'] = -1
            wine_dict['dry_sweet'] = -1
            wine_dict['soft_acidic'] = -1

        # Define wine_dict and assign each element
        wine_dict['winery'] = winery
        wine_dict['wine_type'] = wine_type
        wine_dict['city'] = city
        wine_dict['country'] = country
        wine_dict['wine_name'] = wine_name
        wine_dict['year'] = year
        wine_dict['overall_rating'] = overall_rating
        wine_dict['overall_rating_count'] = overall_rating_count
        wine_dict['price'] = price

        try:
            writer.writerow(wine_dict)
        except Exception as e:
            print(e)
            pass
            
csv_file.close()

Scraping Page Number: 12
Scraping Item Number: 1
Scraping Item Number: 2
Scraping Item Number: 3
Scraping Item Number: 4
Scraping Item Number: 5
Scraping Item Number: 6
Scraping Item Number: 7
Scraping Item Number: 8
Scraping Item Number: 9
Scraping Item Number: 10
Scraping Item Number: 11
Scraping Item Number: 12
Scraping Item Number: 13
Scraping Item Number: 14
Scraping Item Number: 15
Scraping Item Number: 16
Scraping Item Number: 17
Scraping Item Number: 18
Scraping Item Number: 19
Scraping Item Number: 20
Scraping Item Number: 21
Scraping Item Number: 22
Scraping Item Number: 23
Scraping Item Number: 24
Scraping Item Number: 25
Scraping Item Number: 26
Scraping Item Number: 27
Scraping Item Number: 28
Scraping Item Number: 29
Scraping Item Number: 30
Scraping Item Number: 31
Scraping Item Number: 32
Scraping Item Number: 33
Scraping Item Number: 34
Scraping Item Number: 35
Scraping Item Number: 36
Scraping Item Number: 37
Scraping Item Number: 38
Scraping Item Number: 39
Scraping 

list index out of range
Scraping Item Number: 320
Scraping Item Number: 321
Scraping Item Number: 322
Scraping Item Number: 323
Scraping Item Number: 324
Scraping Item Number: 325
Scraping Item Number: 326
Scraping Item Number: 327
Scraping Item Number: 328
Scraping Item Number: 329
Scraping Item Number: 330
Scraping Item Number: 331
Scraping Item Number: 332
Scraping Item Number: 333
Scraping Item Number: 334
Scraping Item Number: 335
Scraping Item Number: 336
Scraping Item Number: 337
Scraping Item Number: 338
Scraping Item Number: 339
Scraping Item Number: 340
Scraping Item Number: 341
Scraping Item Number: 342
Scraping Item Number: 343
list index out of range
Scraping Item Number: 344
Scraping Item Number: 345
Scraping Item Number: 346
Scraping Item Number: 347
Scraping Item Number: 348
Scraping Item Number: 349
Scraping Item Number: 350
Scraping Item Number: 351
Scraping Item Number: 352
Scraping Item Number: 353
Scraping Item Number: 354
Scraping Item Number: 355
Scraping Item Nu

Scraping Item Number: 630
Scraping Item Number: 631
Scraping Item Number: 632
Scraping Item Number: 633
Scraping Item Number: 634
Scraping Item Number: 635
Scraping Item Number: 636
Scraping Item Number: 637
Scraping Item Number: 638
Scraping Item Number: 639
Scraping Item Number: 640
Scraping Item Number: 641
Scraping Item Number: 642
Scraping Item Number: 643
Scraping Item Number: 644
Scraping Item Number: 645
Scraping Item Number: 646
Scraping Item Number: 647
Scraping Item Number: 648
Scraping Item Number: 649
Scraping Item Number: 650
Scraping Item Number: 651
Scraping Item Number: 652
Scraping Item Number: 653
Scraping Item Number: 654
Scraping Item Number: 655
Scraping Item Number: 656
Scraping Item Number: 657
Scraping Item Number: 658
Scraping Item Number: 659
Scraping Item Number: 660
Scraping Item Number: 661
Scraping Item Number: 662
Scraping Item Number: 663
Scraping Item Number: 664
Scraping Item Number: 665
Scraping Item Number: 666
Scraping Item Number: 667
Scraping Ite

Scraping Item Number: 193
Scraping Item Number: 194
Scraping Item Number: 195
Scraping Item Number: 196
Scraping Item Number: 197
Scraping Item Number: 198
Scraping Item Number: 199
Scraping Item Number: 200
Scraping Item Number: 201
Scraping Item Number: 202
Scraping Item Number: 203
Scraping Item Number: 204
Scraping Item Number: 205
Scraping Item Number: 206
Scraping Item Number: 207
Scraping Item Number: 208
Scraping Item Number: 209
Scraping Item Number: 210
Scraping Item Number: 211
Scraping Item Number: 212
Scraping Item Number: 213
Scraping Item Number: 214
Scraping Item Number: 215
Scraping Item Number: 216
Scraping Item Number: 217
Scraping Item Number: 218
Scraping Item Number: 219
Scraping Item Number: 220
Scraping Item Number: 221
Scraping Item Number: 222
Scraping Item Number: 223
Scraping Item Number: 224
Scraping Item Number: 225
Scraping Item Number: 226
Scraping Item Number: 227
Scraping Item Number: 228
Scraping Item Number: 229
Scraping Item Number: 230
Scraping Ite

Scraping Item Number: 509
Scraping Item Number: 510
Scraping Item Number: 511
Scraping Item Number: 512
Scraping Item Number: 513
Scraping Item Number: 514
Scraping Item Number: 515
Scraping Item Number: 516
Scraping Item Number: 517
Scraping Item Number: 518
Scraping Item Number: 519
Scraping Item Number: 520
Scraping Item Number: 521
Scraping Item Number: 522
Scraping Item Number: 523
Scraping Item Number: 524
Scraping Item Number: 525
Scraping Item Number: 526
Scraping Item Number: 527
Scraping Item Number: 528
Scraping Item Number: 529
Scraping Item Number: 530
Scraping Item Number: 531
Scraping Item Number: 532
Scraping Item Number: 533
Scraping Item Number: 534
Scraping Item Number: 535
Scraping Item Number: 536
Scraping Item Number: 537
Scraping Item Number: 538
Scraping Item Number: 539
Scraping Item Number: 540
Scraping Item Number: 541
Scraping Item Number: 542
Scraping Item Number: 543
Scraping Item Number: 544
Scraping Item Number: 545
Scraping Item Number: 546
Scraping Ite

...

Scraping Item Number: 726
Scraping Item Number: 727
Scraping Item Number: 728
Scraping Item Number: 729
Scraping Item Number: 730
Scraping Item Number: 731
Scraping Item Number: 732
Scraping Item Number: 733
Scraping Item Number: 734
Scraping Item Number: 735
Scraping Item Number: 736
Scraping Item Number: 737
Scraping Item Number: 738
Scraping Item Number: 739
Scraping Item Number: 740
Scraping Item Number: 741
Scraping Item Number: 742
Scraping Item Number: 743
Scraping Item Number: 744
'charmap' codec can't encode characters in position 14-18: character maps to <undefined>
Scraping Item Number: 745
Scraping Item Number: 746
Scraping Item Number: 747
Scraping Item Number: 748
Scraping Item Number: 749
Scraping Item Number: 750
Scraping Item Number: 751
Scraping Item Number: 752
Scraping Item Number: 753
Scraping Item Number: 754
Scraping Item Number: 755
Scraping Item Number: 756
Scraping Item Number: 757
Scraping Item Number: 758
Scraping Item Number: 759
Scraping Item Number: 760
S

Scraping Item Number: 16
Scraping Item Number: 17
Scraping Item Number: 18
Scraping Item Number: 19
Scraping Item Number: 20
Scraping Item Number: 21
Scraping Item Number: 22
Scraping Item Number: 23
Scraping Item Number: 24
Scraping Item Number: 25
Scraping Item Number: 26
Scraping Item Number: 27
Scraping Item Number: 28
Scraping Item Number: 29
Scraping Item Number: 30
Scraping Item Number: 31
Scraping Item Number: 32
Scraping Item Number: 33
Scraping Item Number: 34
Scraping Item Number: 35
Scraping Item Number: 36
Scraping Item Number: 37
Scraping Item Number: 38
Scraping Item Number: 39
Scraping Item Number: 40
Scraping Item Number: 41
Scraping Item Number: 42
Scraping Item Number: 43
Scraping Item Number: 44
Scraping Item Number: 45
Scraping Item Number: 46
Scraping Item Number: 47
Scraping Item Number: 48
Scraping Item Number: 49
Scraping Item Number: 50
Scraping Item Number: 51
Scraping Item Number: 52
Scraping Item Number: 53
Scraping Item Number: 54
Scraping Item Number: 55


Scraping Item Number: 331
Scraping Item Number: 332
Scraping Item Number: 333
Scraping Item Number: 334
Scraping Item Number: 335
Scraping Item Number: 336
Scraping Item Number: 337
Scraping Item Number: 338
Scraping Item Number: 339
Scraping Item Number: 340
Scraping Item Number: 341
Scraping Item Number: 342
Scraping Item Number: 343
Scraping Item Number: 344
'charmap' codec can't encode characters in position 21-25: character maps to <undefined>
Scraping Item Number: 345
Scraping Item Number: 346
Scraping Item Number: 347
Scraping Item Number: 348
Scraping Item Number: 349
Scraping Item Number: 350
Scraping Item Number: 351
Scraping Item Number: 352
Scraping Item Number: 353
Scraping Item Number: 354
Scraping Item Number: 355
Scraping Item Number: 356
Scraping Item Number: 357
Scraping Item Number: 358
Scraping Item Number: 359
Scraping Item Number: 360
Scraping Item Number: 361
Scraping Item Number: 362
Scraping Item Number: 363
Scraping Item Number: 364
Scraping Item Number: 365
S

Scraping Item Number: 640
Scraping Item Number: 641
Scraping Item Number: 642
Scraping Item Number: 643
Scraping Item Number: 644
Scraping Item Number: 645
Scraping Item Number: 646
Scraping Item Number: 647
Scraping Item Number: 648
Scraping Item Number: 649
Scraping Item Number: 650
Scraping Item Number: 651
Scraping Item Number: 652
Scraping Item Number: 653
Scraping Item Number: 654
Scraping Item Number: 655
Scraping Item Number: 656
Scraping Item Number: 657
Scraping Item Number: 658
Scraping Item Number: 659
Scraping Item Number: 660
Scraping Item Number: 661
Scraping Item Number: 662
Scraping Item Number: 663
Scraping Item Number: 664
Scraping Item Number: 665
Scraping Item Number: 666
Scraping Item Number: 667
Scraping Item Number: 668
Scraping Item Number: 669
Scraping Item Number: 670
Scraping Item Number: 671
Scraping Item Number: 672
Scraping Item Number: 673
Scraping Item Number: 674
Scraping Item Number: 675
Scraping Item Number: 676
Scraping Item Number: 677
Scraping Ite

Scraping Item Number: 952
Scraping Item Number: 953
Scraping Item Number: 954
Scraping Item Number: 955
Scraping Item Number: 956
Scraping Item Number: 957
Scraping Item Number: 958
Scraping Item Number: 959
Scraping Item Number: 960
Scraping Item Number: 961
Scraping Item Number: 962
Scraping Item Number: 963
Scraping Item Number: 964
Scraping Item Number: 965
Scraping Item Number: 966
Scraping Item Number: 967
Scraping Item Number: 968
Scraping Item Number: 969
Scraping Item Number: 970
Scraping Item Number: 971
Scraping Item Number: 972
Scraping Item Number: 973
Scraping Item Number: 974
Scraping Item Number: 975
Scraping Item Number: 976
Scraping Item Number: 977
Scraping Item Number: 978
Scraping Item Number: 979
Scraping Item Number: 980
Scraping Item Number: 981
Scraping Item Number: 982
Scraping Item Number: 983
Scraping Item Number: 984
Scraping Item Number: 985
Scraping Item Number: 986
Scraping Item Number: 987
Scraping Item Number: 988
Scraping Item Number: 989
Scraping Ite